In [12]:
# The following section is used to generate information about the datasets so these 
# variables can be initialised in the CNN Model. The information gathered here will
# also be used to determine an appropiate size of the training and testing datasets.
# Import os to change the information surrounding the enviroment 
import os                                                  
# Create tensorflow image batches using data augmentation.
from keras.preprocessing.image import ImageDataGenerator    
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
locationOfDataset = '/home/dmr/DatasetForExperiments/ransomware/'
batches = ImageDataGenerator().flow_from_directory(directory=locationOfDataset, 
                                                   target_size=(64,64), 
                                                   batch_size=101)

# Print information about Images 
imgs, labels = next(batches)
print("Number of Images and Size of Images: ",imgs.shape) # Images with width x length x depth
print("Number of Images and Families: ",labels.shape) # Lablels with batch_size, number of classes

Found 101 images belonging to 10 classes.
Number of Images and Size of Images:  (101, 64, 64, 3)
Number of Images and Families:  (101, 10)


In [2]:
# This section will cover the necessary steps used to split the scareware dataset
# into training and testing to evaluate the effectiveness of this convolutional neural 
# network model based on the scareware samples.
import numpy as np                                   # Provides the ability to use numerical data to convert the .dex files to greyscale images
import scipy as sp                                   # Provides the use of algorithms to be used 
from PIL import Image                                # Allow for processing of images

# Splitting the Dataset into Training and testing with a test size of 30% of overall 
# samples in the dataset. 
from sklearn.model_selection import train_test_split    # Used to split the dataset

# Split up the data set, 20% testing 80% training
trainX, testX, trainY, testY = train_test_split(imgs/255.,labels, test_size=0.2)

# Print the information regarding the training and testing sample sizes
#print("Size of Training Set: ", trainX.shape)                   # Print training set information
#print("Size of Testing Set: ",testX.shape)                      # Print testing set information
print("Number of Samples in Training Set: ",trainY.shape)   # Print training set information
print("Number of Samples in Testing Set: ",testY.shape)     # Print testing set information

Number of Samples in Training Set:  (80, 10)
Number of Samples in Testing Set:  (21, 10)


In [3]:
# The Convolutional Neural Network Model
# This section will encompass the aspects surrounding the convultional neural network
# which was used for the experimentation of this research. 
import keras                                            # Import keras
import tensorflow                                       # Import tensorflow
from keras.models import Sequential, Input, Model       # Used for CNN model regarding Models
from keras.layers import Dense, Dropout, Flatten        # Used for CNN model regarding Layers
from keras.layers import Conv2D, MaxPooling2D           # Used for CNN model regarding Layers
from tensorflow.keras.layers import BatchNormalization  # Layer used to normalise inputs
from tensorflow.keras.metrics import AUC                # Used estimate to Area Under Curve

noOfFamilies = 10 # Number of Families in the Sample

# Function used to create the layers of the convolutional neural network model. These parameters were
# constantly changed throughout testing in order to obtain the best possible results. Within this model 
# there are 10 main layers and each layers parameters are explained in detail. 
def CNNModel():
    CNNModel = Sequential()        # Sequential Model is used the experiments

    # Convolutional Layer: With relu Activation, 30 filters, and a kernel size of 3 x 3
    CNNModel.add(Conv2D(31, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))                # Layer 1 

    # Max Pooling Layer: With a pool size of 2 x 2
    CNNModel.add(MaxPooling2D(pool_size=(2, 2)))            # Layer 2
    
    # Convolutional Layer: 16 filters and kernel size of 3 x 3 
    CNNModel.add(Conv2D(16, (3, 3), activation='relu'))     # Layer 3
    
    # Max Pooling Layer: Pool size of 2 x 2 
    CNNModel.add(MaxPooling2D(pool_size=(2, 2)))            # Layer 4
    
    # Drop Out Layer: This layer used to drop 25% of neurons in the model .
    CNNModel.add(Dropout(0.25))                             # Layer 5
    
    # Flatten Layer
    CNNModel.add(Flatten())                                 # Layer 6
    
    # Dense/Fully Connected Layer : 128 Neurons using the relu activation function
    CNNModel.add(Dense(128, activation='relu'))             # Layer 7
    
    # Drop Out Layer: This layer used to drop 50% of neurons in the model.
    CNNModel.add(Dropout(0.5))                              # Layer 8
    
    # Dense/Fully Connected Layer : 50 Neurons using the relu activation function
    CNNModel.add(Dense(50, activation='relu'))              # Layer 9
    
    # Dense/Fully Connected Layer : number of families neurons, using the softmax activation function
    CNNModel.add(Dense(noOfFamilies, activation='softmax')) # Layer 10
    
    CNNModel.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return CNNModel     # Return the Convolutional Neural Network Model

In [4]:
# Measure the effectiveness of the model. This section is used to generate the results of the 
# corresponding convolutional neural network. For the purpose of this mode, there are three main 
# measurements which have to be declared. These are recall, preicision and f1. Listed below
# are the neccessary formulas which are required to obtain these results.
from keras import backend as K

# Recall Measurement
def recallMeasurement(testY, yPrediction):
    true_positives = K.sum(K.round(K.clip(testY * yPrediction, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(testY, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

# Precision Measurement
def precisionMeasurement(testY, yPrediction):
    true_positives = K.sum(K.round(K.clip(testY * yPrediction, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(yPrediction, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# F1 Measurement
def f1Measurement(testY, yPrediction):
    precision = precisionMeasurement(testY, yPrediction)
    recall = recallMeasurement(testY, yPrediction)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [5]:
# Initialise the Model
CNNModel = CNNModel()

2023-05-05 14:53:20.599139: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-05-05 14:53:20.915142: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-05 14:53:20.915191: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-05-05 14:53:20.916057: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 14:53:20.923207: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz
2023-05-05 14:53:20.9237

In [6]:
# Print the Information of the Model. This includes the information surrounding 
# the layers of the models as well as the number of total parameters
CNNModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 31)        868       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 31)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 16)        4480      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [7]:
trainY.shape                            # Returns the shape of the trainY array 
trainY_new = np.argmax(trainY, axis=1)  # Converts the trainY array to an array of integers 
trainY_new                              # Returns the array of integer labels

array([1, 5, 4, 4, 0, 4, 6, 7, 8, 0, 1, 7, 2, 5, 0, 6, 6, 2, 3, 9, 0, 4,
       4, 5, 9, 5, 7, 1, 3, 7, 8, 4, 8, 7, 2, 0, 6, 3, 7, 3, 1, 5, 5, 9,
       7, 5, 8, 1, 2, 9, 6, 9, 4, 1, 6, 0, 6, 5, 8, 9, 7, 1, 2, 2, 8, 5,
       0, 2, 8, 8, 7, 0, 4, 2, 9, 8, 9, 0, 7, 1])

In [8]:
# This section of the model imports the class_weight function from the sklearn.utils 
# library. This is used to compute the class weights for a multi-class classification problem.
# The weightOfModel computes the class weights based on the class distribution found in the 
# training set. The class_weight parameter is set to balanced which means that the class 
# weights are automatically adjusted in acorrdance with the information obtained from the 
# input data.

from sklearn.utils import class_weight # Used to compute the class weights for a multi-class classification system
weightOfModel = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(trainY_new),
                                                 y = trainY_new)

# This code is used to map each unique class label in the training set trainY_new variable. 
# This is then uses the class_weight variable initialised above. 
weightOfModel = {l:c for l,c in zip(np.unique(trainY_new), weightOfModel)}

In [9]:
# This is used to compile the Convolutional Neural network Model. It uses the adam optimiser, 
# computes loss using binary_crossentropy and then calculates the measurements listed before.
CNNModel.compile(optimizer='adam', 
                    loss='binary_crossentropy', 
                    metrics=[f1Measurement,precisionMeasurement, 
                    recallMeasurement,
                    AUC()])

In [10]:
# This section of the model is used to trained the convolutional neural network model. 
# This is achieved by using the fit() method from the Keras API of the TensorFlow library which 
# was imported previously. Here is where the number of epochs are declared for the model also
CNNModel.fit(trainX, trainY, validation_data=(testX, testY), 
                epochs=150,  
                class_weight=weightOfModel)

2023-05-05 14:53:21.093632: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/150
3/3 [==============================] - 1s 272ms/step - loss: 0.6136 - f1Measurement: 0.0000e+00 - precisionMeasurement: 0.0000e+00 - recallMeasurement: 0.0000e+00 - auc: 0.5022 - val_loss: 0.3702 - val_f1Measurement: 0.0000e+00 - val_precisionMeasurement: 0.0000e+00 - val_recallMeasurement: 0.0000e+00 - val_auc: 0.5611
Epoch 2/150
3/3 [==============================] - 0s 54ms/step - loss: 0.4002 - f1Measurement: 0.0150 - precisionMeasurement: 0.0417 - recallMeasurement: 0.0091 - auc: 0.4975 - val_loss: 0.3598 - val_f1Measurement: 0.0000e+00 - val_precisionMeasurement: 0.0000e+00 - val_recallMeasurement: 0.0000e+00 - val_auc: 0.5563
Epoch 3/150
3/3 [==============================] - 0s 60ms/step - loss: 0.3944 - f1Measurement: 0.0658 - precisionMeasurement: 0.1407 - recallMeasurement: 0.0430 - auc: 0.5600 - val_loss: 0.3186 - val_f1Measurement: 0.0000e+00 - val_precisionMeasurement: 0.0000e+00 - val_recallMeasurement: 0.0000e+00 - val_auc: 0.6842
Epoch 4/150
3/3 [==========

Epoch 27/150
3/3 [==============================] - 0s 89ms/step - loss: 0.2193 - f1Measurement: 0.3490 - precisionMeasurement: 0.8843 - recallMeasurement: 0.2188 - auc: 0.9182 - val_loss: 0.2011 - val_f1Measurement: 0.5517 - val_precisionMeasurement: 1.0000 - val_recallMeasurement: 0.3810 - val_auc: 0.9558
Epoch 28/150
3/3 [==============================] - 0s 67ms/step - loss: 0.2144 - f1Measurement: 0.3554 - precisionMeasurement: 0.9327 - recallMeasurement: 0.2279 - auc: 0.9271 - val_loss: 0.1953 - val_f1Measurement: 0.5517 - val_precisionMeasurement: 1.0000 - val_recallMeasurement: 0.3810 - val_auc: 0.9588
Epoch 29/150
3/3 [==============================] - 0s 60ms/step - loss: 0.2033 - f1Measurement: 0.5222 - precisionMeasurement: 0.8944 - recallMeasurement: 0.3711 - auc: 0.9283 - val_loss: 0.1914 - val_f1Measurement: 0.6000 - val_precisionMeasurement: 1.0000 - val_recallMeasurement: 0.4286 - val_auc: 0.9564
Epoch 30/150
3/3 [==============================] - 0s 54ms/step - loss: 

Epoch 80/150
3/3 [==============================] - 0s 54ms/step - loss: 0.0245 - f1Measurement: 0.9823 - precisionMeasurement: 0.9909 - recallMeasurement: 0.9740 - auc: 0.9999 - val_loss: 0.1744 - val_f1Measurement: 0.8095 - val_precisionMeasurement: 0.8095 - val_recallMeasurement: 0.8095 - val_auc: 0.9059
Epoch 81/150
3/3 [==============================] - 0s 54ms/step - loss: 0.0286 - f1Measurement: 0.9696 - precisionMeasurement: 0.9825 - recallMeasurement: 0.9570 - auc: 0.9997 - val_loss: 0.1780 - val_f1Measurement: 0.8293 - val_precisionMeasurement: 0.8500 - val_recallMeasurement: 0.8095 - val_auc: 0.9293
Epoch 82/150
3/3 [==============================] - 0s 55ms/step - loss: 0.0227 - f1Measurement: 0.9831 - precisionMeasurement: 0.9831 - recallMeasurement: 0.9831 - auc: 0.9998 - val_loss: 0.1850 - val_f1Measurement: 0.8293 - val_precisionMeasurement: 0.8500 - val_recallMeasurement: 0.8095 - val_auc: 0.9295
Epoch 83/150
3/3 [==============================] - 0s 54ms/step - loss: 

3/3 [==============================] - 0s 55ms/step - loss: 0.0106 - f1Measurement: 1.0000 - precisionMeasurement: 1.0000 - recallMeasurement: 1.0000 - auc: 1.0000 - val_loss: 0.2044 - val_f1Measurement: 0.7143 - val_precisionMeasurement: 0.7143 - val_recallMeasurement: 0.7143 - val_auc: 0.9276
Epoch 133/150
3/3 [==============================] - 0s 56ms/step - loss: 0.0085 - f1Measurement: 1.0000 - precisionMeasurement: 1.0000 - recallMeasurement: 1.0000 - auc: 1.0000 - val_loss: 0.2093 - val_f1Measurement: 0.7000 - val_precisionMeasurement: 0.7368 - val_recallMeasurement: 0.6667 - val_auc: 0.9064
Epoch 134/150
3/3 [==============================] - 0s 52ms/step - loss: 0.0051 - f1Measurement: 1.0000 - precisionMeasurement: 1.0000 - recallMeasurement: 1.0000 - auc: 1.0000 - val_loss: 0.2146 - val_f1Measurement: 0.7805 - val_precisionMeasurement: 0.8000 - val_recallMeasurement: 0.7619 - val_auc: 0.9075
Epoch 135/150
3/3 [==============================] - 0s 57ms/step - loss: 0.0109 - f

In [11]:
# Evaluates the trained  model on the test data and returns the values of the  metrics
scores = CNNModel.evaluate(testX, testY) 
print("--------------------------------------------------------------------------------------------------------------------")

# Evaluates the trained model on the test data and declares the loss, accuracy 
# f1_score, precision and recall
loss, accuracy, f1_score, precision, recall = CNNModel.evaluate(testX, testY)
print("--------------------------------------------------------------------------------------------------------------------")

# Used to Print the Final CNN Accuracy of the model
print("Final Results for The CNN Model")
print("F1 Accuracy of: {:.2f}".format(scores[1]),'%')    # Print Accuracy of Model
print("With a Loss of: {:.2f}".format(scores[0]),'%')    # Print Loss of Model

1/1 [==============================] - 0s 19ms/step - loss: 0.1951 - f1Measurement: 0.7619 - precisionMeasurement: 0.7619 - recallMeasurement: 0.7619 - auc: 0.8907
--------------------------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 28ms/step - loss: 0.1951 - f1Measurement: 0.7619 - precisionMeasurement: 0.7619 - recallMeasurement: 0.7619 - auc: 0.8907
--------------------------------------------------------------------------------------------------------------------
Final Results for The CNN Model
F1 Accuracy of: 0.76 %
With a Loss of: 0.20 %
